# 랜덤포레스트
- Bagging의 확장판
- Bagging의 데이터 샘플링 방식 + 다수결 / 평균을 사용
- 각 노드에서 분할 할 때 모든 특성(피쳐) 중 일부만 무작위 선택 -> 트리 간의 상관성 낮춰준다. 다양성 증가
- 여러 결정 트리를 서로 다르게 학습 시킨 뒤 투표(분류) 또는 평균(회귀)으로 예측하는 Bagging 기반의 앙상블 모델
- 부트스트랩 샘플링(중복허용)과 특성 무작위 선택으로 모델 간의 상관성을 줄여 성능과 안정성을 높이는 방법
- 고차원 데이터에서 안정성
- 대부분의 실무에서 많이 사용이 되는 알고리즘

- 매개변수
    - n_estimators
        - 기본값: 100
        - 모델의 생성 개수
    - criterion
        - 분류 : gini(기본값), entropy, log_loss
        - 회귀 : squared_error(기본값), absolute_error, friedman_mse, poisson
    - max_depth
        - 기본값 : None
        - 트리의 최대 깊이
    - bootstrap
        - 기본값 : True
        - 트리학습에서 부트스트랩 사용 여부
    - max_features
        - 분류의 기본값 : 'sqrt' (피쳐의 개수의 루트 값)
        - 회귀의 기본값 : 1.0 (모든 피쳐)
        - 가능한 값 : int, float(0~1), 'sqrt', 'log2'
    - min_sample_leaf
        - 기본값 : 1
        - 리프노드의 최소의 샘플 수
        - 값이 크면 -> 리프가 너무 세분화되는 것을 방지하여 과적합 방지
    - max_leaf_node
        - 기본값 : None
        - 트리 전체에서 리프 노드의 개수 제한
        - 너무 많은 리프 생성 -> 과적합
        - 너무 적은 리프 생성 -> 단순화되어 성능 저하
    - min_weight_fraction_leaf
        - 기본값 : 0.0
        - 샘플가중치의 합이 전체 데이터에서 차지하는 최소의 비율
        - 데이터가 불균형 문제를 가지고 있는 경우 값들을 변경하여 유용
    - oob_score
        - 기본값 : False
        - 부트스트랩에서 빠진 샘플을 이용하여 검증 점수를 추정
    - max_samples
        - 기본값 : None
        - bootstrap이 True일 때, 각 트리가 사용할 샘플 수 (비율)
- 속성
    - estimators_
        - 개별 결정트리들의 리스트
    - feature_importances_
        - feature들의 중요도
    - n_feture_in_ / feature_names_in_
        - 입력이 되는 피쳐의 개수 / 입력이 되는 피쳐의 이름들
    - oob_score_
        - OOB 점수 (oob_score 매개변수가 True 일때)
    - oob_decision_function_
        - 분류에서 사용 가능
        - OOB 데이터들의 확률/결정함수
    - oob_prediction_
        - 회귀에서 사용 가능
        - OOB 예측값
- 메서드
    - fit(x, y)
        - 모델의 학습
    - predict(x)
        - 예측값 출력
    - score(x, y)
        - 분류에서는 정확도, 회귀에서는 r2score
    - apply(x)
        - 각 샘플이 각 트리에서 도달하는 리프인덱스를 반환
    - decision_path(x)
        - 트리 내의 경로

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [4]:
body = pd.read_csv("../data_git/data/bodyPerformance.csv")
body['gender'] = np.where(body['gender'] == 'M' , 0,  1)
body['class'] = body['class'].map(
    {
        'A': 0,
        'B' : 1,
        'C' : 2,
        'D' : 3
    }
)
feature_names = body.columns.difference(['class'])
x = body[feature_names].values
y = body['class']
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [5]:
# 랜덤포레스트 모델을 생성
clf = RandomForestClassifier()
clf.fit(X_train, Y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [6]:
pred = clf.predict(X_test)

In [7]:
# 해당 데이터에서 다중 클래스 분류 -> 정밀도, 재현율, f1score를 생성할 때는 average의 값을 변경
cm = confusion_matrix(pred, Y_test)
acc = accuracy_score(pred, Y_test)
prc = precision_score(pred, Y_test, average = 'micro')
rcll = recall_score(pred, Y_test, average = 'micro')
f1 = f1_score(pred, Y_test, average = 'micro')

print(cm)
print("정확도 : ", round(acc, 2))
print("정밀도 : ", round(prc, 2))
print("재현율 : ", round(rcll, 2))
print("f1 : ", round(f1, 2))

[[872 238  99  16]
 [129 590 183  49]
 [  2 136 674 106]
 [  1  40  49 834]]
정확도 :  0.74
정밀도 :  0.74
재현율 :  0.74
f1 :  0.74


In [32]:
# RandomForest 분류 모델의 성능을 개선하기 위해서 하이퍼파라미터를 수정
# 학습할 트리의 개수를 200개 증가
# 노드 분할에 필요한 최소 샘플 수는 기본값 2 -> 4 변환
# 트리 전체에서 리프 노드의 최대 개수의 제한을 10개 변환

clf2 = RandomForestClassifier(
    n_estimators=200,
    min_samples_split=4,
    max_leaf_nodes=500
)
clf2.fit(X_train, Y_train)
print(round(
    clf2.score(X_test, Y_test), 2
    )
)

0.74
